In [1]:
import json
from pprint import pprint
TRAIN_FILE_PATH = "/tmp/semeval24_task3/final_clean_data/train/Subtask_2.json"
VALIDATION_FILE_PATH = "/tmp/semeval24_task3/final_clean_data/val/Subtask_2.json"
with open(TRAIN_FILE_PATH) as f:
    train_data = json.load(f)
with open(VALIDATION_FILE_PATH) as f:
    validation_data = json.load(f)

pprint(len(train_data))
pprint(len(validation_data))

1236
138


In [2]:
class EmotionIndexer:
    def __init__(self):
        self.emotion_to_index = {
            'joy': 0,
            'sadness': 1,
            'anger': 2,
            'neutral': 3,
            'surprise': 4,
            'disgust': 5,
            'fear': 6,
            'pad': 7,
        }
        self.emotion_freq = [0]*7
        self.weights = None

        self.index_to_emotion = {index: emotion for emotion, index in self.emotion_to_index.items()}

    def emotion_to_idx(self, emotion):
        return self.emotion_to_index.get(emotion, None)

    def idx_to_emotion(self, index):
        return self.index_to_emotion.get(index, None)
    
    def compute_weights(self, data):
        for conversation in data:
            conversation = conversation['conversation']
            for utterance in conversation:
                emotion = utterance['emotion']
                self.emotion_freq[self.emotion_to_index[emotion]] += 1
        print(self.emotion_freq)
        self.weights = [1/freq for freq in self.emotion_freq]

# Example usage
indexer = EmotionIndexer()
indexer.compute_weights(train_data)
print(indexer.weights)

[2075, 1008, 1489, 5258, 1670, 376, 340]
[0.00048192771084337347, 0.000992063492063492, 0.000671591672263264, 0.0001901863826550019, 0.0005988023952095808, 0.0026595744680851063, 0.0029411764705882353]


In [3]:
import torch
import json
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_video
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np

/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import pickle

class YourAudioEncoder():
    def __init__(self, audio_embeddings_path):
        with open(audio_embeddings_path, "rb") as f:
            self.audio_embeddings = pickle.load(f)

    def lmao(self, audio_name):
        audio_name = audio_name.split(".")[0]
        audio_embedding = self.audio_embeddings[audio_name]
        return torch.from_numpy(audio_embedding)
    
class YourVideoEncoder():
    def __init__(self, video_embeddings_path):
        with open(video_embeddings_path, "rb") as f:
            self.video_embeddings = pickle.load(f)

    def lmao(self, video_name):
        # video_name = video_name.split(".")[0]
        video_embedding = self.video_embeddings[video_name].reshape((16,-1))
        video_embedding = np.mean(video_embedding, axis=0)
        return torch.from_numpy(video_embedding)

class YourTextEncoder():
    def __init__(self, text_embeddings_path):
        with open(text_embeddings_path, "rb") as f:
            self.text_embeddings = pickle.load(f)

    def lmao(self, video_name):
        text_embedding = self.text_embeddings[video_name]
        return torch.from_numpy(text_embedding)


In [5]:
class ConversationDataset(Dataset):
    def __init__(self, json_file, audio_encoder, video_encoder, text_encoder, max_seq_len):
        self.max_seq_len = max_seq_len
        self.data = self.load_data(json_file)
        self.audio_encoder = audio_encoder
        self.video_encoder = video_encoder
        self.text_encoder = text_encoder

    def load_data(self, json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        conversation = self.data[idx]['conversation']
        emotion_labels = [utterance['emotion'] for utterance in conversation]
        audio_paths = [utterance['video_name'].replace('mp4', 'wav') for utterance in conversation]
        video_paths = [utterance['video_name'] for utterance in conversation]
        texts = [utterance['video_name'] for utterance in conversation]

        audio_embeddings = [self.audio_encoder.lmao(audio_path) for audio_path in audio_paths]
        video_embeddings = [self.video_encoder.lmao(video_path) for video_path in video_paths]
        text_embeddings = [self.text_encoder.lmao(text) for text in texts]

        cause_pairs = self.data[idx]['emotion-cause_pairs']
        useful_utterances = set([int(cause_pair[1]) for cause_pair in cause_pairs])
        cause_labels = []
        for utterance in conversation:
            if utterance['utterance_ID'] in useful_utterances:
                cause_labels.append(1)
            else:
                cause_labels.append(0)
        
        
        # Pad or truncate conversations to the maximum sequence length
        if len(conversation) < self.max_seq_len:
            pad_length = self.max_seq_len - len(conversation)
            audio_embeddings += [torch.zeros_like(audio_embeddings[0])] * pad_length
            video_embeddings += [torch.zeros_like(video_embeddings[0])] * pad_length
            text_embeddings += [torch.zeros_like(text_embeddings[0])] * pad_length
            cause_labels += [-1] * pad_length
            emotion_labels += ['pad'] * pad_length
            pad_mask = [1] * len(conversation) + [0] * pad_length
        else:
            audio_embeddings = audio_embeddings[:self.max_seq_len]
            video_embeddings = video_embeddings[:self.max_seq_len]
            text_embeddings = text_embeddings[:self.max_seq_len]
            emotion_labels = emotion_labels[:self.max_seq_len]
            cause_labels = cause_labels[:self.max_seq_len]
            pad_mask = [1] * self.max_seq_len

        emotion_indices = [indexer.emotion_to_idx(emotion) for emotion in emotion_labels]
        
        audio_embeddings = torch.stack(audio_embeddings)
        video_embeddings = torch.stack(video_embeddings)
        text_embeddings = torch.stack(text_embeddings)
        emotion_indices = torch.from_numpy(np.array(emotion_indices))
        pad_mask = torch.from_numpy(np.array(pad_mask))
        cause_labels = torch.from_numpy(np.array(cause_labels))
        
        return {
            'audio': audio_embeddings,
            'video': video_embeddings,
            'text': text_embeddings,
            'emotion_labels': emotion_indices,
            'pad_mask': pad_mask,
            'cause_labels': cause_labels,
        }
# Example usage
# You need to define your audio, video, and text encoders accordingly

# Define your data paths
DATA_DIR = "/tmp/semeval24_task3"

AUDIO_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "audio_embeddings", "audio_embeddings.pkl")
VIDEO_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "video_embeddings", "train", "video_embeddings.pkl")
TEXT_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "text_embeddings", "text_embeddings.pkl")

audio_encoder = YourAudioEncoder(AUDIO_EMBEDDINGS_FILEPATH)
video_encoder = YourVideoEncoder(VIDEO_EMBEDDINGS_FILEPATH)
text_encoder = YourTextEncoder(TEXT_EMBEDDINGS_FILEPATH)
max_seq_len = 40  # Adjust this according to your needs

# Create the dataset and dataloader
train_dataset = ConversationDataset(TRAIN_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

validation_dataset = ConversationDataset(VALIDATION_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True)

# Example of iterating through batches
# for batch in dataloader:
#     audio = batch['audio']  # Shape: (batch_size, max_seq_len, audio_embedding_size)
#     video = batch['video']  # Shape: (batch_size, max_seq_len, video_embedding_size)
#     text = batch['text']    # Shape: (batch_size, max_seq_len, text_embedding_size)
#     emotions = batch['emotion_labels']  # List of emotion labels for each utterance in the batch


In [6]:
import torch
import torch.nn as nn

class EmotionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, num_emotions, embedding_dropout=0.2):
        super(EmotionClassifier, self).__init__()
        
        self.audio_dropout = nn.Dropout(embedding_dropout)
        self.video_dropout = nn.Dropout(embedding_dropout)
        self.text_dropout = nn.Dropout(embedding_dropout)

        self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)
        self.relu = nn.ReLU()
        
        self.second_linear_layer = nn.Linear(hidden_size, hidden_size, dtype=torch.float32)
        # Replace Transformer with BiLSTM
        self.bilstm = nn.LSTM(hidden_size, hidden_size // 2, num_layers, 
                              dropout=dropout, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size, num_emotions)

    def forward(self, audio_encoding, video_encoding, text_encoding):
        # Concatenate or combine the audio, video, and text encodings
        audio_encoding = audio_encoding.float()
        video_encoding = video_encoding.float()
        text_encoding = text_encoding.float().squeeze()
        
        audio_encoding = self.audio_dropout(audio_encoding)
        video_encoding = self.video_dropout(video_encoding)
        text_encoding = self.text_dropout(text_encoding)
        
        combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
        combined_encoding = self.first_linear(combined_encoding)
        combined_encoding = self.relu(combined_encoding)
        combined_encoding = self.second_linear_layer(combined_encoding)
        
        # Pass through BiLSTM
        lstm_output, _ = self.bilstm(combined_encoding)

        # Take the output of the BiLSTM
        emotion_logits = self.linear(lstm_output)
        # Apply a softmax layer
        emotion_logits = torch.softmax(emotion_logits, dim=2)

        return emotion_logits

In [7]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import get_linear_schedule_with_warmup

# Define your model
# model = EmotionClassifier(input_size=11237, hidden_size=5000, num_layers=2, num_heads=2, dropout=0.2, num_emotions=7)
emotion_model = EmotionClassifier(input_size=768*3, hidden_size=2000, num_layers=3, dropout=0.6, num_emotions=7)
cause_model = EmotionClassifier(input_size=768*3, hidden_size=2000, num_layers=2, dropout=0.6, num_emotions=2)
emotion_model.to("cuda")
cause_model.to("cuda")

weights_tensor = torch.tensor(np.array(indexer.weights)).to("cuda").float()
emotion_criterion = nn.CrossEntropyLoss(
    weight=weights_tensor,
    ignore_index=7
)

cause_criterion = nn.CrossEntropyLoss(ignore_index=-1)

num_epochs = 30
total_steps = len(train_dataloader) * num_epochs

emotion_optimizer = AdamW(emotion_model.parameters(), lr=0.0001)
emotion_lr_scheduler = get_linear_schedule_with_warmup(
    emotion_optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

cause_optimizer = AdamW(cause_model.parameters(), lr=1e-4)
cause_lr_scheduler = get_linear_schedule_with_warmup(
    cause_optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


# Define training parameters

# Training loop
for epoch in (range(num_epochs)):
    emotion_model.train()  # Set the model to training mode
    cause_model.train()
    
    total_loss = 0.0
    total_tokens = 0
    total_correct_emotions = 0
    total_correct_causes = 0
    total_predictions = 0

    for batch in tqdm(train_dataloader):  # Assuming you have a DataLoader for your dataset
        # Extract data from the batch
        audio = batch['audio'].to('cuda')
        video = batch['video'].to('cuda')
        text = batch['text'].to('cuda')
        cause_indices = batch['cause_labels'].to('cuda')
        
        audio_copy = audio.clone().detach()
        video_copy = video.clone().detach()
        text_copy = text.clone().detach()
        
        emotion_indices = batch['emotion_labels'].to('cuda')
        pad_mask = batch['pad_mask'].to('cuda')

        # Forward pass
        emotion_logits = emotion_model(audio_copy, video_copy, text_copy)

        # Reshape emotion_logits
        emotion_logits = emotion_logits.view(-1, emotion_logits.size(-1))

        # Flatten emotion_indices (assuming it's a 2D tensor with shape [batch_size, max_sequence_length])
        emotion_indices = emotion_indices.view(-1)

        # Calculate a mask to exclude padded positions from the loss
        pad_mask = pad_mask.view(-1).float()

        # Calculate the loss, excluding padded positions
        emotion_loss = emotion_criterion(emotion_logits, emotion_indices)
        # masked_loss = torch.sum(loss * pad_mask) / torch.sum(pad_mask)
        masked_loss = emotion_loss# *pad_mask
        # Backpropagation and optimization
        
        
        cause_logits = cause_model(audio, video, text)
        cause_logits = cause_logits.view(-1, cause_logits.size(-1))
        cause_indices = cause_indices.view(-1)
        
        cause_loss = cause_criterion(cause_logits, cause_indices)
        masked_loss += cause_loss
        
        emotion_optimizer.zero_grad()
        cause_optimizer.zero_grad()
        
        masked_loss.backward()
        
        emotion_optimizer.step()
        cause_optimizer.step()        

        total_loss += masked_loss.item()
        total_tokens += torch.sum(pad_mask).item()
        
        predicted_emotions = torch.argmax(emotion_logits, dim=1)
        correct_predictions_emotions = ((predicted_emotions == emotion_indices) * pad_mask).sum().item()

        predicted_causes = torch.argmax(cause_logits, dim=1)
        correct_predictions_causes = ((predicted_causes == cause_indices) * pad_mask).sum().item()
        
        total_correct_emotions += correct_predictions_emotions
        total_correct_causes += correct_predictions_causes
        total_predictions += torch.sum(pad_mask).item()  # Batch size
        
    
    emotion_lr_scheduler.step()
    cause_lr_scheduler.step()
    
    emotion_model.eval()  # Set the model to evaluation mode
    cause_model.eval()
    
    total_val_loss = 0.0
    total_val_tokens = 0
    total_val_correct_emotions = 0
    total_val_correct_causes = 0
    total_val_predictions = 0
    true_labels_emotion = []
    predicted_labels_emotion = []
    true_labels_cause = []
    predicted_labels_cause = []
    padded_labels = []

    with torch.no_grad():
        for val_batch in tqdm(validation_dataloader):
            audio = val_batch['audio'].to('cuda')
            video = val_batch['video'].to('cuda')
            text = val_batch['text'].to('cuda')
            emotion_indices = val_batch['emotion_labels'].to('cuda')
            cause_indices = val_batch['cause_labels'].to('cuda')
            pad_mask = val_batch['pad_mask'].to('cuda')
            
            audio_copy = audio.clone().detach()
            video_copy = video.clone().detach()
            text_copy = text.clone().detach()

            emotion_logits = emotion_model(audio_copy, video_copy, text_copy)

            # Reshape emotion_logits
            emotion_logits = emotion_logits.view(-1, emotion_logits.size(-1))

            # Flatten emotion_indices (assuming it's a 2D tensor with shape [batch_size, max_sequence_length])
            emotion_indices = emotion_indices.view(-1)

            pad_mask = pad_mask.view(-1)   

            # Calculate the loss, excluding padded positions
            val_loss = emotion_criterion(emotion_logits, emotion_indices)
            masked_loss = val_loss #torch.sum(val_loss * pad_mask) / torch.sum(pad_mask)
            
            cause_logits = cause_model(audio, video, text)
            cause_logits = cause_logits.view(-1, cause_logits.size(-1))
            cause_indices = cause_indices.view(-1)
            cause_loss = cause_criterion(cause_logits, cause_indices)
            masked_loss += cause_loss
            
            total_val_loss += masked_loss.item()
            total_val_tokens += torch.sum(pad_mask).item()
            
            predicted_emotions_val = torch.argmax(emotion_logits, dim=1)
            correct_predictions_val = ((predicted_emotions_val == emotion_indices) * pad_mask).sum().item()
            total_val_correct_emotions += correct_predictions_val
            
            predicted_causes_val = torch.argmax(cause_logits, dim=1)
            correct_predictions_causes_val = ((predicted_causes_val == cause_indices) * pad_mask).sum().item()
            total_val_correct_causes += correct_predictions_causes_val
            
            total_val_predictions += torch.sum(pad_mask).item()

            # Store true and predicted labels for F1 score calculation
            true_labels_emotion.extend(emotion_indices.cpu().numpy())
            predicted_labels_emotion.extend(predicted_emotions_val.cpu().numpy())
            
            true_labels_cause.extend(cause_indices.cpu().numpy())
            predicted_labels_cause.extend(predicted_causes_val.cpu().numpy())
            padded_labels.extend(pad_mask.cpu().numpy())

    final_true_labels_emotion = [label for label, pad in zip(true_labels_emotion, padded_labels) if pad == 1]
    final_predicted_labels_emotion = [label for label, pad in zip(predicted_labels_emotion, padded_labels) if pad == 1]
    
    final_true_labels_cause = [label for label, pad in zip(true_labels_cause, padded_labels) if pad == 1]
    final_predicted_labels_cause = [label for label, pad in zip(predicted_labels_cause, padded_labels) if pad == 1]
    
    emotion_classification_rep = classification_report(final_true_labels_emotion, final_predicted_labels_emotion)
    cause_classification_rep = classification_report(final_true_labels_cause, final_predicted_labels_cause)
    
    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / total_tokens
    avg_val_loss = total_val_loss / total_val_tokens
    
    print("===============================")
    print("Training data metrics")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Training Loss: {avg_loss}")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Accuracy: {total_correct_emotions / total_predictions}")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Accuracy: {total_correct_causes / total_predictions}")
    
    print("VALIDATION METRICS")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Validation Loss: {avg_val_loss}")
    print(emotion_classification_rep)
    print(cause_classification_rep)
    print("===============================")

    torch.save(emotion_model.state_dict(), f"/tmp/semeval24_task3/final_models/emotion_models/emotion_model_{epoch:02}.pt")
    torch.save(cause_model.state_dict(), f"/tmp/semeval24_task3/final_models/cause_models/cause_model_{epoch:02}.pt")

print("Training complete!")


100%|██████████| 9/9 [00:00<00:00, 10.20it/s]
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use

Training data metrics
Epoch [1/30] Training Loss: 0.016602963326253686
Epoch [1/30] Accuracy: 0.12459070072036674
Epoch [1/30] Accuracy: 0.5833333333333334
VALIDATION METRICS
Epoch [1/30] Validation Loss: 0.01626831484282773
              precision    recall  f1-score   support

           0       0.37      0.32      0.35       226
           1       0.11      0.91      0.19       139
           2       0.12      0.02      0.03       126
           3       0.33      0.00      0.01       671
           4       0.00      0.00      0.00       170
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        33

    accuracy                           0.15      1403
   macro avg       0.13      0.18      0.08      1403
weighted avg       0.24      0.15      0.08      1403

              precision    recall  f1-score   support

           0       0.69      0.70      0.69       795
           1       0.60      0.58      0.59       608

    accuracy  

100%|██████████| 9/9 [00:00<00:00, 10.15it/s]
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/akshett.jindal/miniconda3/envs/semeval24-task3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use

Training data metrics
Epoch [2/30] Training Loss: 0.01596294467403909
Epoch [2/30] Accuracy: 0.2767681728880157
Epoch [2/30] Accuracy: 0.6635559921414538
VALIDATION METRICS
Epoch [2/30] Validation Loss: 0.015792115959879848
              precision    recall  f1-score   support

           0       0.39      0.39      0.39       226
           1       0.20      0.62      0.30       139
           2       0.35      0.23      0.28       126
           3       0.61      0.39      0.48       671
           4       0.11      0.12      0.12       170
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        33

    accuracy                           0.35      1403
   macro avg       0.24      0.25      0.22      1403
weighted avg       0.42      0.35      0.36      1403

              precision    recall  f1-score   support

           0       0.76      0.62      0.68       795
           1       0.60      0.75      0.67       608

    accuracy   

100%|██████████| 9/9 [00:00<00:00, 10.48it/s]


...

Training data metrics
Epoch [28/30] Training Loss: 0.012304722852931743
Epoch [28/30] Accuracy: 0.6074001309757695
Epoch [28/30] Accuracy: 0.9237066142763589
VALIDATION METRICS
Epoch [28/30] Validation Loss: 0.015218304636133457
              precision    recall  f1-score   support

           0       0.49      0.57      0.53       226
           1       0.38      0.41      0.39       139
           2       0.28      0.24      0.26       126
           3       0.74      0.62      0.68       671
           4       0.51      0.60      0.55       170
           5       0.07      0.16      0.10        38
           6       0.15      0.18      0.16        33

    accuracy                           0.53      1403
   macro avg       0.38      0.40      0.38      1403
weighted avg       0.56      0.53      0.54      1403

              precision    recall  f1-score   support

           0       0.71      0.70      0.71       795
           1       0.62      0.63      0.62       608

    accura

100%|██████████| 9/9 [00:00<00:00, 10.15it/s]


Training data metrics
Epoch [29/30] Training Loss: 0.01230332754996374
Epoch [29/30] Accuracy: 0.6150130975769482
Epoch [29/30] Accuracy: 0.923215455140799
VALIDATION METRICS
Epoch [29/30] Validation Loss: 0.015353747460983178
              precision    recall  f1-score   support

           0       0.49      0.53      0.51       226
           1       0.51      0.33      0.40       139
           2       0.28      0.40      0.33       126
           3       0.74      0.60      0.66       671
           4       0.50      0.58      0.54       170
           5       0.05      0.13      0.08        38
           6       0.13      0.21      0.16        33

    accuracy                           0.52      1403
   macro avg       0.39      0.40      0.38      1403
weighted avg       0.57      0.52      0.54      1403

              precision    recall  f1-score   support

           0       0.72      0.67      0.70       795
           1       0.61      0.66      0.63       608

    accuracy

100%|██████████| 9/9 [00:00<00:00, 10.41it/s]


Training data metrics
Epoch [30/30] Training Loss: 0.012175424901503867
Epoch [30/30] Accuracy: 0.6268009168303864
Epoch [30/30] Accuracy: 0.9300098231827112
VALIDATION METRICS
Epoch [30/30] Validation Loss: 0.015228467419244354
              precision    recall  f1-score   support

           0       0.42      0.54      0.47       226
           1       0.34      0.50      0.41       139
           2       0.29      0.32      0.30       126
           3       0.79      0.45      0.58       671
           4       0.43      0.62      0.51       170
           5       0.06      0.18      0.09        38
           6       0.12      0.12      0.12        33

    accuracy                           0.47      1403
   macro avg       0.35      0.39      0.36      1403
weighted avg       0.56      0.47      0.49      1403

              precision    recall  f1-score   support

           0       0.74      0.63      0.68       795
           1       0.59      0.71      0.64       608

    accura